In [1]:
import sys
sys.path.append('/home/cadeniran/storage/cadeniran/mpp/memscan-local')

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import re
import sys
import matplotlib.pyplot as plt
from pyrosetta import *
from modules import AH_CA_rmsd
from modules import HelixTools

In [3]:
init()

┌──────────────────────────────────────────────────────────────────────────────┐
│                                 PyRosetta-4                                  │
│              Created in JHU by Sergey Lyskov and PyRosetta Team              │
│              (C) Copyright Rosetta Commons Member Institutions               │
│                                                                              │
│ NOTE: USE OF PyRosetta FOR COMMERCIAL PURPOSES REQUIRE PURCHASE OF A LICENSE │
│         See LICENSE.PyRosetta.md or email license@uw.edu for details         │
└──────────────────────────────────────────────────────────────────────────────┘
PyRosetta-4 2024 [Rosetta PyRosetta4.Release.python311.ubuntu 2024.39+release.59628fbc5bc09f1221e1642f1f8d157ce49b1410 2024-09-23T07:49:48] retrieved from: http://www.pyrosetta.org
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.Release.python311.ubuntu r387 2024.39+release.59628fbc5b 59628fbc5b

In [3]:
plt.rcParams['figure.figsize'] = 5,5
plt.rcParams['axes.linewidth'] = 2.0
plt.rcParams['savefig.dpi'] = 500
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
np.set_printoptions(legacy='1.25')

### Functions

In [5]:
class HelixTools:
    """ Some python functions to calculate helix parameters """    
    def __init__(self):
        self.diff_vec = None
        self.degree = 0

 
    def calculate_screw_axis(self, Pose):
        """ In order to calculate the normal, you need to first calculate the list of 
        alpha carbons. Because the last residue of a membrane-initiated pose is the
        membrane itself, the last protein residue is the n-1th residue. The screw axis
        is calculated as the average coordinates of the helix residues 2-4 and (n-1)-(n-4).
        Note tht this won't work for helices shorter than 7 amino acids. """
        #copy the pose and remove virtual residues
        pose_clone = Pose.clone()
        pyrosetta.rosetta.core.pose.remove_nonprotein_residues(pose_clone)
        #calculate alpha carbon centroids for the first and last 4 residues
        ca_list = []
        for resno in range(1,pose_clone.size()+1):
            ca_list.append(pose_clone.residue(resno).xyz('CA'))
        first = ca_list[1:4]
        last = ca_list[pose_clone.size()-4:pose_clone.size()-1]
        
        average_first = average_vectors(first)        
        average_last = average_vectors(last)
        
        self.screw_vector = average_last - average_first
        if self.screw_vector.is_zero() is False:
            self.screw_vector.normalize()
            
        return self.screw_vector
        
    
    def calc_angle(self, v1, axis):
        """ Calculate the angle between a vector and an axis unit vector """
        if axis == 'x':
            v2 = pyrosetta.rosetta.numeric.xyzVector_double_t(1,0,0)
        elif axis == 'y':
            v2 = pyrosetta.rosetta.numeric.xyzVector_double_t(0,1,0)
        elif axis == 'z':
            v2 = pyrosetta.rosetta.numeric.xyzVector_double_t(0,0,1)
        else:
            print("Enter 'x', 'y', or 'z'")
        
        v1_mag = np.linalg.norm(v1)    
        v2_mag = np.linalg.norm(v2)
        v1_v2_dot = np.dot(v1,v2)
        v1_v2_mag_dot = np.dot(v1_mag, v2_mag)
        
        costh = v1_v2_dot / v1_v2_mag_dot
        self.degree = np.arccos(costh) * 57.2958 #radian to degree conversion

        return self.degree

In [6]:
def AH_CA_rmsd(native, best):
    number_of_residues = native.size()
    pymol = PyMOLMover()

    #initiate the spanless membrane mover
    fm = AddSpanlessMembraneMover()
    fm.add_membrane_virtual(native)
    fm.apply(native)
    #pymol.apply(native)

    #calculate the center of mass for the best and native helices
    cmass_best = pyrosetta.rosetta.core.pose.center_of_mass(best, 1, best.size()-1)
    cmass_native = pyrosetta.rosetta.core.pose.center_of_mass(native, 1, native.size()-1)

    print("The best cmass is: {}".format(cmass_best))
    print("The native cmass is: {}".format(cmass_native))

    move_xy = pyrosetta.rosetta.numeric.xyzVector_double_t(-cmass_native[0], -cmass_native[1],0)

    #run the translation
    shifted = native.clone()
    copy_best = best.clone()
    translation_mover = pyrosetta.rosetta.protocols.rigid.WholeBodyTranslationMover(move_xy)
    translation_mover.apply(shifted)

    print(pyrosetta.rosetta.core.pose.center_of_mass(shifted, 1, shifted.size()-1))

    #calculate the helix parameters
    ht = HelixTools()
    helix_normal = ht.calculate_screw_axis(shifted)
    native_angle_with_x = ht.calc_angle(helix_normal,'x')
    native_angle_with_y = ht.calc_angle(helix_normal,'y')
    native_angle_with_z = ht.calc_angle(helix_normal,'z')
    print(native_angle_with_x, native_angle_with_y, native_angle_with_z)

    helix_best_normal = ht.calculate_screw_axis(best)
    best_angle_with_x = ht.calc_angle(helix_best_normal,'x')
    best_angle_with_y = ht.calc_angle(helix_best_normal,'y')
    best_angle_with_z = ht.calc_angle(helix_best_normal,'z')
    print(best_angle_with_x, best_angle_with_y, best_angle_with_z)

    x_diff = best_angle_with_x - native_angle_with_x
    print(x_diff)

    if cmass_best[2] > 0:
        if x_diff < 0:
            align_x = pyrosetta.rosetta.protocols.rigid.WholeBodyRotationMover(pyrosetta.rosetta.numeric.xyzVector_double_t(0,0,1), pyrosetta.rosetta.core.pose.center_of_mass(shifted, 1, shifted.size()-1), -x_diff)
            align_x.apply(shifted)
        else:
            align_x = pyrosetta.rosetta.protocols.rigid.WholeBodyRotationMover(pyrosetta.rosetta.numeric.xyzVector_double_t(0,0,1), pyrosetta.rosetta.core.pose.center_of_mass(shifted, 1, shifted.size()-1), x_diff)
            align_x.apply(shifted)

    else:
        if x_diff > 0:
            align_x = pyrosetta.rosetta.protocols.rigid.WholeBodyRotationMover(pyrosetta.rosetta.numeric.xyzVector_double_t(0,0,1), pyrosetta.rosetta.core.pose.center_of_mass(shifted, 1, shifted.size()-1), -x_diff)
            align_x.apply(shifted)
        else:
            align_x = pyrosetta.rosetta.protocols.rigid.WholeBodyRotationMover(pyrosetta.rosetta.numeric.xyzVector_double_t(0,0,1), pyrosetta.rosetta.core.pose.center_of_mass(shifted, 1, shifted.size()-1), x_diff)
            align_x.apply(shifted)


    helix_normal = ht.calculate_screw_axis(shifted)
    native_angle_with_x = ht.calc_angle(helix_normal,'x')
    native_angle_with_y = ht.calc_angle(helix_normal,'y')
    native_angle_with_z = ht.calc_angle(helix_normal,'z')
    print(native_angle_with_x, native_angle_with_y, native_angle_with_z)    

    y_diff = best_angle_with_y - native_angle_with_y

    align_y = pyrosetta.rosetta.protocols.rigid.WholeBodyRotationMover(pyrosetta.rosetta.numeric.xyzVector_double_t(0,0,1), pyrosetta.rosetta.core.pose.center_of_mass(shifted, 1, shifted.size()-1), -y_diff)
    align_y.apply(shifted) 

    helix_native_normal = ht.calculate_screw_axis(shifted)
    native_angle_with_x = ht.calc_angle(helix_native_normal,'x')
    native_angle_with_y = ht.calc_angle(helix_native_normal,'y')
    native_angle_with_z = ht.calc_angle(helix_native_normal,'z')
    
    x_diff = native_angle_with_x - best_angle_with_x
    y_diff = native_angle_with_y - best_angle_with_y
    z_diff = native_angle_with_z - best_angle_with_z

    print(x_diff, y_diff, z_diff)

    shifted.dump_pdb("shifted_native.pdb")
    #pymol.apply(shifted)
 
    ca_shifted = []
    ca_best = []

    #calculate the CA positions for the shifted original and the best structures
    for i in range(1,shifted.size()):
        ca_shifted.append(np.array(shifted.residue(i).xyz('CA')))
    for j in range(1, best.size()):
        ca_best.append(np.array(copy_best.residue(j).xyz('CA')))
  
    shifted_df = pd.DataFrame(ca_shifted)
    best_df = pd.DataFrame(ca_best)

    total = 0
    for k in range(number_of_residues):
        if all(item < 0 for item in shifted_df[2]) is all(item < 0 for item in best_df[2]):
            total = total + np.square((ca_shifted[k][0] - ca_best[k][0])) + np.square((ca_shifted[k][1] - ca_best[k][1])) + np.square((ca_shifted[k][2] - ca_best[k][2]))
        else:
            total = total + np.square((abs(ca_shifted[k][0]) - abs(ca_best[k][0]))) + np.square((abs(ca_shifted[k][1]) - abs(ca_best[k][1]))) + np.square((abs(ca_shifted[k][2]) - abs(ca_best[k][2])))

    average = total/len(ca_best)
    squared_average = np.sqrt(average)

    return squared_average

## Define lists and dictionary

In [7]:
subdir=[]
proteinlist=[]

dir='/home/cadeniran/storage/cadeniran/mpp/memscan-local/results_grid'
dirlist = [ item for item in os.listdir(dir) if os.path.isdir(os.path.join(dir, item)) ]
for dir in dirlist:
    subdir.append(dir)
subdir.sort()

dir='/home/cadeniran/storage/cadeniran/mpp/memscan-local/results_grid/1.0_15/results'
dirlist = [ item for item in os.listdir(dir) if os.path.isdir(os.path.join(dir, item)) ]
for dir in dirlist:
    proteinlist.append(dir)
proteinlist.sort()

In [19]:
depthlist = []
rmsdlist=[]
tiltlist=[]

length = len(proteinlist)
for i in range(length):
    protein = proteinlist[i]
    protein_tag = protein.split(sep='_')[0]
    multiple_tag = protein.split(sep='_')[1]

    os.chdir("/home/cadeniran/storage/cadeniran/mpp/memscan-local/results_grid/1.0_15/")
    native = pose_from_pdb('input_pdbs/{}_{}_renum.pdb'.format(protein_tag,multiple_tag))
    best = pose_from_pdb('results/{}_{}/output_pdbs/{}_{}_renum_best_pose_overall.pdb'.format(protein_tag,multiple_tag,protein_tag,multiple_tag))

    ### DEPTH CALC
    regexp = re.compile(r'The best depth.*?([0-9.-]+)')
    with open('results/{}_{}/txt/{}_{}_renum_best_scores.txt'.format(protein_tag,multiple_tag,protein_tag,multiple_tag)) as f:
        for line in f:
            match = regexp.match(line)
            if match:
                #print(match.group(1))
                depthlist.append(match.group(1))


    ### RMSD CALC
    rmsd = AH_CA_rmsd(native, best)
    rmsdlist.append(rmsd)


    ### TILT CALC
    v1 = HelixTools().calculate_screw_axis(best)
    bestdegree = HelixTools().calc_angle(v1, 'z')

    v11 = HelixTools().calculate_screw_axis(native)
    nativedegree = HelixTools().calc_angle(v11, 'z')

    diff = bestdegree - nativedegree
    tiltlist.append(diff)


### Build dataframe and save data
df = pd.DataFrame(columns=['PDB ID', 'RMSD', 'Depth', 'Tilt'])
df['PDB ID'] = proteinlist
df['RMSD'] = rmsdlist
df['Depth'] = depthlist
df['Tilt'] = tiltlist
df.to_csv('data.csv', sep=',')

core.import_pose.import_pose: File 'input_pdbs/1bwn_A_renum.pdb' automatically determined to be of type PDB
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] PDB reader is ignoring atom  CD  in residue 18 A.  Pass flag -ignore_zero_occupancy false to change this behavior
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] PDB reader is ignoring atom  CE  in residue 18 A.  Pass flag -ignore_zero_occupancy false to change this behavior
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] PDB reader is ignoring atom  NZ  in residue 18 A.  Pass flag -ignore_zero_occupancy false to change this behavior
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] PDB reader is ignoring atom  NZ  in residue 26 A.  Pass flag -ignore_zero_occupancy false to change this behavior
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] PDB reader is ignoring atom  CZ  in residue 45 A.  Pass flag -ignore_zero_occupancy false to change this behavior
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING 

core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] PDB reader is ignoring atom  CD  in residue 298 A.  Pass flag -ignore_zero_occupancy false to change this behavior
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] PDB reader is ignoring atom  CE  in residue 298 A.  Pass flag -ignore_zero_occupancy false to change this behavior
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] PDB reader is ignoring atom  NZ  in residue 298 A.  Pass flag -ignore_zero_occupancy false to change this behavior
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] PDB reader is ignoring atom  CB  in residue 308 A.  Pass flag -ignore_zero_occupancy false to change this behavior
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] PDB reader is ignoring atom  CG  in residue 308 A.  Pass flag -ignore_zero_occupancy false to change this behavior
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] PDB reader is ignoring atom  CD  in residue 308 A.  Pass flag -ignore_zero_occupancy false to change 

core.pack.pack_missing_sidechains: packing residue number 26 because of missing atom number 9 atom name  NZ
core.pack.pack_missing_sidechains: packing residue number 45 because of missing atom number 9 atom name  CZ
core.pack.pack_missing_sidechains: packing residue number 47 because of missing atom number 8 atom name  NE
core.pack.pack_missing_sidechains: packing residue number 59 because of missing atom number 8 atom name  CE
core.pack.pack_missing_sidechains: packing residue number 83 because of missing atom number 6 atom name  OG
core.pack.pack_missing_sidechains: packing residue number 84 because of missing atom number 6 atom name  CG
core.pack.pack_missing_sidechains: packing residue number 85 because of missing atom number 6 atom name  CG
core.pack.pack_missing_sidechains: packing residue number 118 because of missing atom number 7 atom name  CD
core.pack.pack_missing_sidechains: packing residue number 119 because of missing atom number 8 atom name  NE
core.pack.pack_missing_sid

core.conformation.Conformation: [ WARNING ] missing heavyatom:  CG  on residue LEU 689
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CD1 on residue LEU 689
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CD2 on residue LEU 689
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OG1 on residue THR 690
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CG2 on residue THR 690
core.conformation.Conformation: [ WARNING ] missing heavyatom:  O   on residue CYS:CtermProteinFull 751
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue CYS:CtermProteinFull 751
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] can't find pose atom for file-residue 688 atom 1OCT (trying to store temperature in PDBInfo)
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] can't find pose atom for file-residue 688 atom 2OCT (trying to store temperature in PDBInfo)
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] can't fin

core.conformation.Conformation: [ WARNING ] missing heavyatom:  OE2 on residue GLU 292
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CD  on residue LYS 295
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CE  on residue LYS 295
core.conformation.Conformation: [ WARNING ] missing heavyatom:  NZ  on residue LYS 295
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CD  on residue LYS 296
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CE  on residue LYS 296
core.conformation.Conformation: [ WARNING ] missing heavyatom:  NZ  on residue LYS 296
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CG  on residue GLU 300
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CD  on residue GLU 300
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OE1 on residue GLU 300
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OE2 on residue GLU 300
core.conformation.Conformation: [ WARNING ]

...

core.conformation.Conformation: Found disulfide between residues 948 1096
core.conformation.Conformation: Found disulfide between residues 1101 1253
WELCOME TO THE WORLD OF MEMBRANE PROTEINS...

Setting initial membrane center and normal to position used by the user-provided membrane residue
82.96881288302511 65.93693515346025 154.80377267652153
73.85417136393434 64.62107368519867 149.27545161984605
74.68043591926886 65.93693515346028 150.9314246822736
74.68043591926896 65.9369351534602 150.9314246822736
73.07404474544694 67.10967339195219 150.93142468227364
core.import_pose.import_pose: File 'input_pdbs/6zhh_A_renum.pdb' automatically determined to be of type PDB
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue THR:CtermProteinFull 883
core.import_pose.import_pose: File 'results/6zhh_A/output_pdbs/6zhh_A_renum_best_pose_overall.pdb' automatically determined to be of type PDB
WELCOME TO THE WORLD OF MEMBRANE PROTEINS...

Setting initial membrane center and

core.conformation.Conformation: [ WARNING ] missing heavyatom:  CG  on residue GLU:NtermProteinFull 1
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CD  on residue GLU:NtermProteinFull 1
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OE1 on residue GLU:NtermProteinFull 1
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OE2 on residue GLU:NtermProteinFull 1
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CG  on residue LEU 987
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CD1 on residue LEU 987
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CD2 on residue LEU 987
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CG  on residue ARG 989
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CD  on residue ARG 989
core.conformation.Conformation: [ WARNING ] missing heavyatom:  NE  on residue ARG 989
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CZ  on

core.pack.pack_missing_sidechains: packing residue number 267 because of missing atom number 6 atom name  CG1
core.pack.pack_missing_sidechains: packing residue number 269 because of missing atom number 6 atom name  CG
core.pack.pack_missing_sidechains: packing residue number 270 because of missing atom number 6 atom name  CG
core.pack.pack_missing_sidechains: packing residue number 310 because of missing atom number 6 atom name  CG
core.pack.pack_missing_sidechains: packing residue number 360 because of missing atom number 7 atom name  CG
core.pack.task: Packer task: initialize from command line()
core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
core.pack.pack_rotamers: built 179 rotamers at 9 positions.
core.pack.interaction_graph.interaction_graph_factory: Instantiating DensePDInteractionGraph
core.import_pose.import_pose: File 'results/7wc8_A/output_pdbs/7wc8_A_renum_best_pose_overall.pdb' automatically determined to be of type PDB
core.conformation.Conformation: Found dis

core.pack.pack_missing_sidechains: packing residue number 265 because of missing atom number 6 atom name  CG
core.pack.pack_missing_sidechains: packing residue number 296 because of missing atom number 6 atom name  CG
core.pack.pack_missing_sidechains: packing residue number 348 because of missing atom number 6 atom name  CG
core.pack.pack_missing_sidechains: packing residue number 360 because of missing atom number 6 atom name  CG
core.pack.pack_missing_sidechains: packing residue number 392 because of missing atom number 6 atom name  CG
core.pack.pack_missing_sidechains: packing residue number 396 because of missing atom number 6 atom name  CG
core.pack.pack_missing_sidechains: packing residue number 415 because of missing atom number 6 atom name  CG
core.pack.pack_missing_sidechains: packing residue number 417 because of missing atom number 6 atom name  CG
core.pack.task: Packer task: initialize from command line()
core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
core.pack.

core.conformation.Conformation: [ WARNING ] missing heavyatom:  CD  on residue LYS 456
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CE  on residue LYS 456
core.conformation.Conformation: [ WARNING ] missing heavyatom:  NZ  on residue LYS 456
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue ARG:CtermProteinFull 721
core.conformation.Conformation: Found disulfide between residues 70 146
core.conformation.Conformation: Found disulfide between residues 73 145
core.conformation.Conformation: Found disulfide between residues 76 153
core.conformation.Conformation: Found disulfide between residues 237 240
core.conformation.Conformation: Found disulfide between residues 310 375
core.conformation.Conformation: Found disulfide between residues 421 481
core.conformation.Conformation: Found disulfide between residues 521 595
core.conformation.Conformation: Found disulfide between residues 648 703
core.pack.pack_missing_sidechains: packing residue num

## Plots

# 1eak

In [18]:
os.chdir("/home/cadeniran/u1/amphiscan-hmom/")

In [19]:
pdb="1eak"
native = pose_from_pdb("shifted/shift_%s.pdb" % pdb) 
best = pose_from_pdb("results/%s_A/output_pdbs/%s_A_renum_best_pose_overall.pdb" % (pdb, pdb))

core.import_pose.import_pose: File 'shifted/shift_1eak.pdb' automatically determined to be of type PDB
core.conformation.Conformation: Found disulfide between residues 29 34
core.conformation.Conformation: Found disulfide between residues 202 228
core.conformation.Conformation: Found disulfide between residues 216 243
core.conformation.Conformation: Found disulfide between residues 260 286
core.conformation.Conformation: Found disulfide between residues 274 301
core.conformation.Conformation: Found disulfide between residues 318 344
core.conformation.Conformation: Found disulfide between residues 332 359
core.import_pose.import_pose: File 'results/1eak_A/output_pdbs/1eak_A_renum_best_pose_overall.pdb' automatically determined to be of type PDB
core.conformation.Conformation: Found disulfide between residues 29 34
core.conformation.Conformation: Found disulfide between residues 202 228
core.conformation.Conformation: Found disulfide between residues 216 243
core.conformation.Conformatio

In [20]:
pyrosetta.rosetta.core.pose.full_model_info.make_sure_full_model_info_is_setup(native)
pyrosetta.rosetta.core.pose.full_model_info.make_sure_full_model_info_is_setup(best)
rmsd = pyrosetta.rosetta.protocols.stepwise.modeler.align.superimpose_with_stepwise_aligner(native, best)
print(rmsd)
#native.dump_pdb("shifted.pdb")

protocols.stepwise.modeler.align.StepWisePoseAligner: RMSD 0.000 (0 atoms in ), superimposed on 1267 atoms in 1-422 (RMSD 2.9766031)
2.9766030882730137


In [23]:
AH_CA_rmsd(native, best)

0.20561880397728316

# 1fao

In [24]:
os.chdir("/home/cadeniran/u1/amphiscan-hmom/")

In [25]:
pdb="1fao"
native = pose_from_pdb("shifted/shift_%s.pdb" % pdb) 
best = pose_from_pdb("results/%s_A/output_pdbs/%s_A_renum_best_pose_overall.pdb" % (pdb, pdb))

core.import_pose.import_pose: File 'shifted/shift_1fao.pdb' automatically determined to be of type PDB
core.conformation.Conformation: Found disulfide between residues 66 79
core.import_pose.import_pose: File 'results/1fao_A/output_pdbs/1fao_A_renum_best_pose_overall.pdb' automatically determined to be of type PDB
core.conformation.Conformation: Found disulfide between residues 66 79


In [26]:
pyrosetta.rosetta.core.pose.full_model_info.make_sure_full_model_info_is_setup(native)
pyrosetta.rosetta.core.pose.full_model_info.make_sure_full_model_info_is_setup(best)
rmsd = pyrosetta.rosetta.protocols.stepwise.modeler.align.superimpose_with_stepwise_aligner(native, best)
print(rmsd)
#native.dump_pdb("shifted.pdb")

protocols.stepwise.modeler.align.StepWisePoseAligner: RMSD 0.000 (0 atoms in ), superimposed on 304 atoms in 1-101 (RMSD 1.3310295)
1.3310295382188255


In [27]:
AH_CA_rmsd(native, best)

0.3754000122068335

# 1vfy

In [28]:
os.chdir("/home/cadeniran/u1/amphiscan-hmom/")

In [29]:
pdb="1vfy"
native = pose_from_pdb("shifted/shift_%s.pdb" % pdb) 
best = pose_from_pdb("results/%s_A/output_pdbs/%s_A_renum_best_pose_overall.pdb" % (pdb, pdb))

core.import_pose.import_pose: File 'shifted/shift_1vfy.pdb' automatically determined to be of type PDB
core.import_pose.import_pose: File 'results/1vfy_A/output_pdbs/1vfy_A_renum_best_pose_overall.pdb' automatically determined to be of type PDB


In [30]:
pyrosetta.rosetta.core.pose.full_model_info.make_sure_full_model_info_is_setup(native)
pyrosetta.rosetta.core.pose.full_model_info.make_sure_full_model_info_is_setup(best)
rmsd = pyrosetta.rosetta.protocols.stepwise.modeler.align.superimpose_with_stepwise_aligner(native, best)
print(rmsd)
#native.dump_pdb("shifted.pdb")

protocols.stepwise.modeler.align.StepWisePoseAligner: RMSD 0.000 (0 atoms in ), superimposed on 205 atoms in 1-68 (RMSD 7.3515146)
7.351514645628948


In [31]:
AH_CA_rmsd(native, best)

1.3960458565837162

# 4gw3

In [32]:
os.chdir("/home/cadeniran/u1/amphiscan-hmom/")

In [33]:
pdb="4gw3"
native = pose_from_pdb("shifted/shift_%s.pdb" % pdb) 
best = pose_from_pdb("results/%s_A/output_pdbs/%s_A_renum_best_pose_overall.pdb" % (pdb, pdb))

core.import_pose.import_pose: File 'shifted/shift_4gw3.pdb' automatically determined to be of type PDB
core.import_pose.import_pose: File 'results/4gw3_A/output_pdbs/4gw3_A_renum_best_pose_overall.pdb' automatically determined to be of type PDB


In [34]:
pyrosetta.rosetta.core.pose.full_model_info.make_sure_full_model_info_is_setup(native)
pyrosetta.rosetta.core.pose.full_model_info.make_sure_full_model_info_is_setup(best)
rmsd = pyrosetta.rosetta.protocols.stepwise.modeler.align.superimpose_with_stepwise_aligner(native, best)
print(rmsd)
#native.dump_pdb("shifted.pdb")

protocols.stepwise.modeler.align.StepWisePoseAligner: RMSD 0.000 (0 atoms in ), superimposed on 856 atoms in 1-285 (RMSD 4.3140506)
4.314050568529694


In [35]:
AH_CA_rmsd(native, best)

0.3276663698486736

In [ ]:
AH_CA_rmsd(native, best)

# 6ei6

In [36]:
os.chdir("/home/cadeniran/u1/amphiscan-hmom/")

In [37]:
pdb="6ei6"
native = pose_from_pdb("shifted/shift_%s.pdb" % pdb) 
best = pose_from_pdb("results/%s_A/output_pdbs/%s_A_renum_best_pose_overall.pdb" % (pdb, pdb))

core.import_pose.import_pose: File 'shifted/shift_6ei6.pdb' automatically determined to be of type PDB
core.conformation.Conformation: Found disulfide between residues 1 8
core.import_pose.import_pose: File 'results/6ei6_A/output_pdbs/6ei6_A_renum_best_pose_overall.pdb' automatically determined to be of type PDB
core.conformation.Conformation: Found disulfide between residues 1 8


In [38]:
pyrosetta.rosetta.core.pose.full_model_info.make_sure_full_model_info_is_setup(native)
pyrosetta.rosetta.core.pose.full_model_info.make_sure_full_model_info_is_setup(best)
rmsd = pyrosetta.rosetta.protocols.stepwise.modeler.align.superimpose_with_stepwise_aligner(native, best)
print(rmsd)
#native.dump_pdb("shifted.pdb")

protocols.stepwise.modeler.align.StepWisePoseAligner: RMSD 0.000 (0 atoms in ), superimposed on 730 atoms in 1-243 (RMSD 3.4220306)
3.4220306465054047


In [39]:
AH_CA_rmsd(native, best)

0.31242148117351143